In [10]:
import time
import ccxt
import pandas as pd

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time

from bs4 import BeautifulSoup
import re
import numpy as np
import os

import backtrader as bt
import math

In [ ]:
## 导入数据
file_path = r'C:\Users\Admin\Desktop\backtrader\datas\binance_BTCUSDT.csv'
file_path = r'C:\Users\Admin\Desktop\backtrader\datas\binance_BTCUSDT.csv'
file_path = '../../datas/binance_BTCUSDT.csv'
funding_data = pd.read_csv(file_path)
file_path = '../../datas/币安USDT资费.csv'
borrow_data = pd.read_csv(file_path)
borrow_data = borrow_data.drop_duplicates(subset='时间')
# 将时间列转换为时间戳
funding_data['datetime'] = pd.to_datetime(funding_data['datetime']).astype('int64') // 10**9
borrow_data['时间'] = pd.to_datetime(borrow_data['时间']).astype('int64') // 10**9

funding_data = funding_data.sort_values(by='datetime', ascending=True).reset_index(drop=True)
borrow_data = borrow_data.sort_values(by='时间', ascending=True).reset_index(drop=True)

# 统一时间间隔为8小时
# 先找出最早和最晚的时间
min_time = max(borrow_data['时间'].min(), funding_data['datetime'].min())
max_time = min(borrow_data['时间'].max(), funding_data['datetime'].max())

# 生成8小时间隔的时间序列
time_series = pd.date_range(start=pd.Timestamp.fromtimestamp(min_time), end=pd.Timestamp.fromtimestamp(max_time), freq='8H').astype('int64') // 10**9

# 对funding_data进行重采样
funding_data.set_index('datetime', inplace=True)
funding_data = funding_data.reindex(time_series).reset_index()

# 对borrow_data进行重采样
borrow_data.set_index('时间', inplace=True)
borrow_data = borrow_data.reindex(time_series).reset_index()

# 合并两个DataFrame
merged_data = pd.concat([funding_data, borrow_data], axis=1)

columns_to_fill = ['年利率', '日利率', '小时利率'] # 简单前向填充，再删除缺失数据
merged_data[columns_to_fill] = merged_data[columns_to_fill].fillna(method='ffill', limit=2)
merged_data = merged_data.dropna()
merged_data = merged_data.drop('Unnamed: 0', axis=1)

# 去除列名重复的列（保留第一个出现的列名）
merged_data = merged_data.loc[:, ~merged_data.columns.duplicated(keep='first')]
merged_data = merged_data.rename(columns={'日利率': 'financing_rate'})
merged_data.to_csv('../../datas/binance_BTCUSDT.csv')
print(merged_data)

           index  fundingRate     年利率  financing_rate      小时利率
2     1690963200     0.000100  0.0486          0.0001  0.000006
3     1690992000     0.000100  0.0486          0.0001  0.000006
4     1691020800     0.000100  0.0486          0.0001  0.000006
5     1691049600     0.000100  0.0510          0.0001  0.000006
6     1691078400     0.000100  0.0510          0.0001  0.000006
...          ...          ...     ...             ...       ...
1839  1743868800     0.000004  0.0582          0.0002  0.000007
1840  1743897600     0.000002  0.0582          0.0002  0.000007
1841  1743926400     0.000054  0.0555          0.0002  0.000006
1842  1743955200    -0.000006  0.0555          0.0002  0.000006
1843  1743984000     0.000049  0.0555          0.0002  0.000006

[1842 rows x 5 columns]


In [11]:
import backtrader as bt
import pandas as pd
import math

class FundingRateArbitrageStrategy(bt.Strategy):
    params = dict(
        leverage_spot=1,  # 现货杠杆倍数
        leverage_contract=1,  # 合约杠杆倍数
        stake=1,  # 交易数量
        printout=True,
        r_s=0.001,  # 现货手续费率
        r_c=0.001,  # 合约手续费率
        r_sl=0.001,  # 滑点率
        r_target=0.05,  # 预期收益率
        rf_threshold=0.001,  # 资金费率阈值
        consecutive_periods=3  # 资金费率连续低于阈值的周期数
    )

    def log(self, txt, dt=None):
        if self.p.printout:
            dt = dt or self.data.datetime[0]
            dt = bt.num2date(dt)
            print('%s, %s' % (dt.isoformat(), txt))

    def notify_order(self, order):
        if order.status in [bt.Order.Submitted, bt.Order.Accepted]:
            return  # Await further notifications

        if order.status == order.Completed:
            if order.isbuy():
                buytxt = 'BUY COMPLETE, %.2f' % order.executed.price
                self.log(buytxt, order.executed.dt)
            else:
                selltxt = 'SELL COMPLETE, %.2f' % order.executed.price
                self.log(selltxt, order.executed.dt)

        elif order.status in [order.Expired, order.Canceled, order.Margin]:
            self.log('%s ,' % order.Status[order.status])
            pass  # Simply log

        # Allow new orders
        self.orderid = None

    def __init__(self):
        # To control operation entries
        self.orderid = None
        self.funding_rate = self.data.funding_rate  # 合约资费数据
        self.financing_rate = self.data.financing_rate  # 融资利率数据
        self.counter = 0  # 资金费率连续低于阈值的计数器
        self.prev_rf_sign = None  # 上一周期资金费率的符号
        self.T = 0  # 持仓周期数量
        self.C = self.broker.getvalue()  # 初始资金
        self.C1 = (self.p.leverage_contract * self.C) / (self.p.leverage_spot + self.p.leverage_contract)  # 现货保证金
        self.C2 = (self.p.leverage_spot * self.C) / (self.p.leverage_spot + self.p.leverage_contract)  # 合约保证金
        self.V_s = (self.p.leverage_spot * self.p.leverage_contract * self.C) / (self.p.leverage_spot + self.p.leverage_contract)  # 现货价值
        self.V_c = (self.p.leverage_spot * self.p.leverage_contract * self.C) / (self.p.leverage_spot + self.p.leverage_contract)  # 合约价值
        self.P_s1 = None  # 现货买入价
        self.P_c1 = None  # 合约开仓价

    def next(self):
        if self.orderid:
            return  # if an order is active, no new orders are allowed

        if self.position.size == 0:
            # 开仓操作
            self.log('BUY CREATE (Spot), %.2f' % self.data.close[0])
            self.buy(size=self.p.stake * self.p.leverage_spot)  # 现货购买
            self.P_s1 = self.data.close[0]

            self.log('SELL CREATE (Contract), %.2f' % self.data.close[0])
            self.sell(size=self.p.stake * self.p.leverage_contract)  # 合约开空
            self.P_c1 = self.data.close[0]
            self.T = 0
            self.counter = 0
            self.prev_rf_sign = math.copysign(1, self.funding_rate[0])

        else:
            # 持仓操作
            self.T += 1
            P_s2 = self.data.close[0]
            P_c2 = self.data.close[0]
            r_f = self.funding_rate[0]
            r_m = self.financing_rate[0]

            # 计算现货收益
            financing_cost = 0
            for i in range(self.T):
                financing_cost += r_m * (1 / (365 * 3))
            R_s = self.V_s * ((P_s2 / self.P_s1) - 1) - self.V_s * (self.p.r_s + self.p.r_sl) - (self.V_s - self.C1) * financing_cost

            # 计算合约收益
            R_c = self.V_c * (1 - (P_c2 / self.P_c1)) - self.V_c * (self.p.r_c + self.p.r_sl)

            # 计算总收益
            funding_income = 0
            for i in range(self.T):
                funding_income += self.funding_rate[i]
            R = R_s + R_c + self.V_c * funding_income

            # 出场条件判断
            if R >= self.C * self.p.r_target:
                self.log('REACH TARGET RETURN, SELL (Spot), %.2f' % self.data.close[0])
                self.sell(size=self.p.stake * self.p.leverage_spot)  # 现货卖出
                self.log('REACH TARGET RETURN, BUY (Contract), %.2f' % self.data.close[0])
                self.buy(size=self.p.stake * self.p.leverage_contract)  # 合约平仓
            elif abs(r_f) < self.p.rf_threshold:
                self.counter += 1
                if self.counter >= self.p.consecutive_periods:
                    self.log('FUNDING RATE BELOW THRESHOLD, SELL (Spot), %.2f' % self.data.close[0])
                    self.sell(size=self.p.stake * self.p.leverage_spot)  # 现货卖出
                    self.log('FUNDING RATE BELOW THRESHOLD, BUY (Contract), %.2f' % self.data.close[0])
                    self.buy(size=self.p.stake * self.p.leverage_contract)  # 合约平仓
            else:
                self.counter = 0

            current_rf_sign = math.copysign(1, r_f)
            if current_rf_sign != self.prev_rf_sign:
                self.log('FUNDING RATE SIGN REVERSED, SELL (Spot), %.2f' % self.data.close[0])
                self.sell(size=self.p.stake * self.p.leverage_spot)  # 现货卖出
                self.log('FUNDING RATE SIGN REVERSED, BUY (Contract), %.2f' % self.data.close[0])
                self.buy(size=self.p.stake * self.p.leverage_contract)  # 合约平仓
            self.prev_rf_sign = current_rf_sign

    def stop(self):
        print('==================================================')
        print('Starting Value - %.2f' % self.C)
        print('Ending   Value - %.2f' % self.broker.getvalue())
        print('==================================================')


if __name__ == '__main__':
    cerebro = bt.Cerebro()

    # 加载包含所有数据的 CSV 文件
    data = pd.read_csv('all_data.csv')
    data['Date'] = pd.to_datetime(data['Date'])
    data.set_index('Date', inplace=True)

    # 创建自定义数据加载器
    class CustomData(bt.feeds.PandasData):
        lines = ('funding_rate', 'financing_rate',)
        params = (
            ('funding_rate', -1),
            ('financing_rate', -1),
        )

    bt_data = CustomData(dataname=data)
    cerebro.adddata(bt_data)

    # 添加策略
    cerebro.addstrategy(FundingRateArbitrageStrategy, leverage_spot=2, leverage_contract=2)

    # 设置初始资金
    cerebro.broker.setcash(100000.0)

    # 设置交易手续费
    cerebro.broker.setcommission(commission=0.001)

    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # 运行回测
    cerebro.run()

    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

FileNotFoundError: [Errno 2] No such file or directory: 'all_data.csv'

In [5]:
# 判断入场条件
def check_entry_condition(data, period_count=3, funding_threshold=0.01,
                          top_n=5, volume_threshold=1000):
    """
    检查是否满足入场条件
    :param data: 历史数据 DataFrame
    :param period_count: 资金费率连续周期数
    :param funding_threshold: 资金费率阈值
    :param top_n: 每层选取的币的数量
    :param volume_threshold: 成交量阈值，用于排除流动性差的币
    :return: 满足入场条件返回 True 和选取的币列表，否则返回 False 和空列表
    """
    # 按照市值分层次，这里简单分为高、中、低三层
    data['market_cap_level'] = pd.qcut(data['market_cap'], q=3, labels=['high','medium', 'low'])

    selected_coins = []
    for level in ['high','medium', 'low']:
        level_data = data[data['market_cap_level'] == level]
        sorted_data = level_data[level_data['volume'] > volume_threshold].sort_values(
            by='annual_return', ascending=False)
        selected_coins.extend(sorted_data.head(top_n)['symbol'].tolist())

    if not selected_coins:
        return False, []

    # 辅助判断资金费率条件
    for i in range(len(data) - period_count + 1):
        sub_data = data.iloc[i:i + period_count]['funding_rate']
        if all(sub_data > funding_threshold) or all(sub_data < -funding_threshold):
            return True, selected_coins

    return False, []


def manage_position(total_funds, leverage):
    """
    仓位管理函数，只考虑杠杆倍数相同的情况
    :param total_funds: 总资金
    :param leverage: 杠杆倍数
    :return: 现货保证金、合约保证金、现货数量、合约数量
    """
    spot_margin = total_funds * 0.5
    contract_margin = total_funds * 0.5
    spot_amount = spot_margin * leverage
    contract_amount = contract_margin * leverage
    return spot_margin, contract_margin, spot_amount, contract_amount


# 仓位管理

In [ ]:
def fill_missing_with_mean(df, col):
    """
    改进版的三等分点均值填充函数
    逻辑：在每两个非缺失值之间，计算三等分点并用两端均值填充
    """
    # 获取非缺失值的索引
    non_missing = df[col].dropna()
    if len(non_missing) < 2:
        return df  # 至少需要两个点才能计算
    
    # 转换为数值索引（如果索引不是数字）
    df = df.reset_index(drop=True) if not df.index.is_numeric() else df
    
    for i in range(len(non_missing) - 1):
        start_idx = non_missing.index[i]
        end_idx = non_missing.index[i+1]
        
        # 计算间隔和填充点
        interval = end_idx - start_idx
        if interval <= 1:  # 相邻点不需要填充
            continue
            
        # 计算三等分点位置
        fill_positions = [
            start_idx + (interval // 3),
            start_idx + 2 * (interval // 3)
        ]
        
        # 计算两端均值
        mean_val = (df.at[start_idx, col] + df.at[end_idx, col]) / 2
        
        # 填充有效位置
        for pos in fill_positions:
            if pd.isna(df.at[pos, col]):  # 只有确实缺失才填充
                df.at[pos, col] = mean_val
                
    return df

# 使用示例
'''
columns_to_fill = ['年利率', '日利率', '小时利率']
for col in columns_to_fill:
    if col in merged_data.columns:
        merged_data = fill_missing_with_mean(merged_data.copy(), col)  # 使用copy避免链式赋值警告;
'''

